Used the original corpus generator from TA Paul because Kevin's didn't work on my machine

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import docx2txt
import pandas as pd
#set working directory
os.chdir('.\Corpus_Txt_Files')
#function to retreive and turn document into text
def retrieve_DSI(file_name):
    file_name=str(file_name)
    text = docx2txt.process(file_name)
    return text
#Lists to store file name and body of text
file_name=[]
text=[]
#for loop to iterate through documents in working directory
for file in os.listdir('.'):
    #if statment to not attempt to open non word documents
    if file.endswith('.docx'):
        text_name=file
        #call function to obtain the text
        text_body=retrieve_DSI(file)
        #apped the file names and text to list
        file_name.append(text_name)
        text.append(text_body)
        #removed the variables used in the for loop
        del text_name, text_body, file
#create dictionary for corpus
corpus={'DSI_Title':file_name, 'Text': text}
#output a CSV with containing the class corpus along with titles of corpus.  
#file saved in working directory.
pd.DataFrame(corpus).to_csv('Corpus.csv', index=file_name)


In [2]:
os.chdir('C:\\Users\\Michael Chyan\\Documents\\MSDS 453\\MSDS453_GroupProject')

In [3]:
import openai
import pandas as pd
import json

I got training data from a Kaggle competititon that can be found here: https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset?select=Twitter_Data.csv

I chose to only use the Twitter dataset and ignored the Reddit data since it matched up with what Kevin was doing

In [4]:
reddit_df = pd.read_csv('Reddit_Data.csv', names=['prompt', 'completion'], encoding='ISO-8859-1')
# Visualize first 5 rows
reddit_df = reddit_df.iloc[1: , :]
reddit_df.head()
reddit_df.head()

,prompt,completion
1,family mormon have never tried explain them t...,1
2,buddhism has very much lot compatible with chr...,1
3,seriously don say thing first all they won get...,-1
4,what you have learned yours and only yours wha...,0
5,for your own benefit you may want read living ...,1


In [5]:
def convert_labels(completion):
  if completion == "0":
    return 'Neutral'
  if completion == "1":
    return 'Positive'
  if completion == "-1":
    return 'Negative'


In [6]:
reddit_df['completion'] = reddit_df['completion'].apply(convert_labels)

reddit_df.head()


,prompt,completion
1,family mormon have never tried explain them t...,Positive
2,buddhism has very much lot compatible with chr...,Positive
3,seriously don say thing first all they won get...,Negative
4,what you have learned yours and only yours wha...,Neutral
5,for your own benefit you may want read living ...,Positive


In [7]:
# emotion_df.to_csv('cleaned.csv', index=False)  

Was planning to fine tune the model using https://beta.openai.com/docs/guides/fine-tuning as a reference but might pivot to using the code block 2 below and hopefully insert in the training data there.

In [8]:
reddit_df.to_json("reddit.jsonl", orient='records', lines=True)


In [9]:
!openai tools fine_tunes.prepare_data -f reddit.jsonl -q


Analyzing...

- Your file contains 37249 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 449 duplicated prompt-completion sets. These are rows: [375, 392, 605, 617, 651, 1222, 1528, 1613, 1760, 1835, 1858, 1900, 1975, 2339, 2422, 2446, 2588, 2780, 2796, 2877, 2924, 2928, 3252, 3307, 3420, 3448, 3544, 3550, 3617, 3649, 3939, 3987, 4219, 4292, 4396, 4400, 4415, 4417, 4431, 4453, 4462, 4632, 4786, 5127, 6248, 6778, 6825, 6885, 6952, 7059, 7420, 7685, 7695, 7713, 7857, 8046, 8129, 8179, 8201, 8968, 9014, 9218, 9219, 9297, 9318, 9806, 10258, 10323, 10597, 10618, 10621, 10626, 10630, 10640, 10643, 10662, 10665, 10666, 10678, 10679, 10684, 10699, 10704, 10726, 10929, 11118, 11131, 11311, 11446,

C:\Users\Michael Chyan\anaconda3\lib\site-packages\openai\validators.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["prompt"] += suffix
C:\Users\Michael Chyan\anaconda3\lib\site-packages\openai\validators.py:421: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["completion"] = x["completion"].apply(


In [10]:
!openai --api-key "API_KEY" api fine_tunes.create -t "reddit_prepared_train.jsonl" -v "reddit_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " Positive" -m ada


^C


!openai api fine_tunes.results -i ft-2zaA7qi0rxJduWQpdvOvmGn3 > result.csv

In [ ]:
emotion_df = pd.read_csv('Twitter_Data.csv', names=['prompt', 'completion'], encoding='ISO-8859-1')
# Visualize first 5 rows
emotion_df = emotion_df.iloc[1: , :]

emotion_df.head()

In [ ]:
emotion_df['completion'] = emotion_df['completion'].apply(convert_labels)

emotion_df.head()
emotion_df.shape

In [ ]:
emotion_df.to_json("emotion.jsonl", orient='records', lines=True)


In [ ]:
!openai tools fine_tunes.prepare_data -f emotion.jsonl -q


In [ ]:
!openai --api-key "API_KEY" api fine_tunes.create -t "emotion_prepared_train.jsonl" -v "emotion_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " Positive" -m ada


In [ ]:
!openai --api-key "API_KEY" api fine_tunes.follow -i ft-IrhvrED2zAUrgvmdCwwBfmlu


In [ ]:
openai.api_key = "API_KEY"
response = openai.Completion.create(
engine="text-davinci-002", # this is the most costly model, might switch to curie for similar performance, less cost
prompt="This is a Sentence sentiment classifier\n\n\nSentence: \"I loved the new Batman movie!\"\nSentiment: Positive\n###\nSentence: \"I hate it when my phone battery dies.\"\nSentiment: Negative\n###\nSentence: \"My day has been 👍\"\nSentiment: Positive\n###\nSentence: \"This is the link to the article\"\nSentiment: Neutral\n###\nSentence: \"This new music video blew my mind\"\nSentiment:",
temperature=0.3,
max_tokens=60,
top_p=1.0,
frequency_penalty=0.5,
presence_penalty=0.0,
stop=["###"])
print (response)

In [ ]:
# import jsonlines
# with jsonlines.open('train.jsonl', mode='w') as writer:
#   for row in emotion_df.itertuples():
#     writer.write({
#       'text': row[1],
#       'label': row[2]
#     })

In [ ]:
# import openai
# openai.api_key='API_KEY'
# result = openai.File.create(file=open("train.jsonl", errors='ignore'), purpose="classifications")

In [ ]:
# filename = result['id']
# openai.Classification.create(
#   search_model="ada",
#   model="curie",
#   file=filename,
#   query="Crypto is crashing hard",
#   labels=["Positive", "Negative", "Neutral"],
# )
# predicted_label = prediction['label']
# print('Predicted label is: {}'.format(predicted_label))